## **範例：多AI代理人預訂飯店**

在當今快速變化的世界中，規劃商務旅行不僅僅是訂機票和飯店房間。它需要達到一種協調與效率的層次，這是相當具有挑戰性的。這正是多AI代理人發揮作用的地方，徹底改變了我們管理旅行需求的方式。

想像你擁有一支智慧代理人團隊，彼此協作，精確且輕鬆地處理你旅程的每個環節。憑藉我們先進的AI技術，我們創建了專門處理預訂服務和行程安排的代理人，確保旅程無阻且無壓力。

這是一個基本場景。規劃商務旅行時，我們需要諮詢商務旅行代理人以取得機票資訊、飯店資訊等。透過AI代理人，我們可以建立訂房服務代理人和行程安排代理人共同合作，提升智慧化水平。


# 初始化 Azure AI Agent 服務並從 **.env** 獲取配置信息

### **.env** 

建立一個 .env 檔案

**.env** 包含 Azure AI Agent 服務的連接字串、AOAI 使用的模型，以及對應的 Google API 搜尋服務 API、ENDPOINT 等。

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "您的 Azure AI Agent 服務模型部署名稱"

[**NOTE**] 您需要使用具備 100,000 速率限制（每分鐘代幣數）及速率限制 600（每分鐘請求數）的模型

  您可以在 Microsoft Foundry - 模型與端點中取得模型。

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "您的 Azure AI Agent 服務專案連接字串"

  您可以在 AI Foundry 入口網站的專案總覽頁面中取得專案連接字串。

- **SERPAPI_SEARCH_API_KEY** = "您的 SERPAPI 搜尋 API KEY"
- **SERPAPI_SEARCH_ENDPOINT** = "您的 SERPAPI 搜尋 Endpoint"

若要取得 Azure AI Agent 服務的模型部署名稱和專案連接字串，需要先建立 Azure AI Agent 服務。建議直接使用[此範本](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json)來建立（***注意：*** Azure AI Agent 服務目前僅在有限區域設定，建議參考[此連結](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support)設定區域）

Agent 需要存取 SERPAPI，建議使用[此連結](https://serpapi.com/searches)註冊。註冊後，您可以取得唯一的 API KEY 和 ENDPOINT。


# Login to Azure 

您現在需要登入 Azure，請在 VScode 中開啟終端機並執行 `az login` 指令


# Setup 

To run this notebook, you will need to install the following libraries. Here is a list of the required libraries and the corresponding pip install commands:

azure-identity: 用於 Azure 驗證。
requests: 用於發送 HTTP 請求。
semantic-kernel: 用於語義內核框架（假設這是一個自訂或特定的庫，您可能需要從特定來源或存儲庫安裝）。


In [ ]:
!pip install azure-identity
!pip install requests
!pip install semantic-kernel
!pip install --upgrade semantic_kernel
!pip install azure-cli

# Explanation: 
import asyncio：這會導入 asyncio 模組，它提供了 Python 中非同步程式設計的支援。它允許你使用 async 和 await 語法來撰寫並發代碼。
from typing import Annotated：這會從 typing 模組中導入 Annotated 類型。Annotated 用於在型別提示中添加元資料，這對於驗證、文件編寫或工具支援等多種用途非常有用。


In [ ]:
import asyncio,os
from typing import Annotated

# 解釋：
透過使用 from dotenv import load_dotenv 以及 load_dotenv()，你可以輕鬆管理配置設定和敏感資訊（例如 API 金鑰和資料庫 URL）在 .env 檔案中，將它們與你的原始碼分離，使你的應用程式更安全且更容易配置。


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Explanation:

Import Statement: from azure.identity.aio import DefaultAzureCredential: 這會從 azure.identity.aio 模組中匯入 DefaultAzureCredential 類別。模組名稱中的 aio 部分表示它是為非同步操作設計的。

Purpose of DefaultAzureCredential: DefaultAzureCredential 類別是 Azure SDK for Python 的一部分。它提供一種預設方式來對 Azure 服務進行身份驗證。它會嘗試按照特定順序使用多種方法進行身份驗證，例如環境變數、管理身分和 Azure CLI 憑證。

Asynchronous Operations: aio 模組表示 DefaultAzureCredential 類別支援非同步操作。這表示你可以使用它與 asyncio 一起執行非阻塞的身份驗證請求。


In [ ]:
from azure.identity.aio import DefaultAzureCredential

# 說明：
從 semantic_kernel 套件匯入各種模組和類別。以下是每個匯入的說明：

AgentGroupChat 來自 semantic_kernel.agents：此類別負責處理與 AI 代理群組聊天相關的功能。AzureAIAgent 及 AzureAIAgentSettings 來自 semantic_kernel.agents.azure_ai

AzureAIAgent：此類別用於建立和管理利用 Azure AI 服務的 AI 代理。

AzureAIAgentSettings：此類別用於配置 AzureAIAgent 的設定。TerminationStrategy 來自 semantic_kernel.agents.strategies.termination.termination_strategy：

此類別定義在特定條件下終止 AI 代理執行的策略。ChatMessageContent 來自 semantic_kernel.contents.chat_message_content：

此類別用來處理聊天訊息的內容。
AuthorRole 來自 semantic_kernel.contents.utils.author_role：

此類別定義在聊天訊息上下文中作者的不同角色。

kernel_function 來自 semantic_kernel.functions.kernel_function_decorator：這個裝飾器用來定義 kernel 函數，即可以在 semantic kernel 框架中執行的函數。
這些匯入設定了創建和管理能在群組聊天環境中互動的 AI 代理所需的組件，可能用於訂房或類似的任務。


In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.contents import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# Explanation:
Next we import the CodeInterpreterTool class from the azure.ai.projects.models module. 

CodeInterpreterTool: 此類別是 Azure AI SDK 的一部分，用於在 AI 專案的上下文中解釋和執行程式碼。它提供執行程式碼片段、分析程式碼或在 AI 工作流程中整合程式碼執行的功能。
此匯入設定了使用 CodeInterpreterTool 所需的元件，對於需要動態解釋和執行程式碼的任務將非常有用。


In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

# 說明：
ApprovalTerminationStrategy 類別提供了一種特定的策略，用於終止 AI 代理的運作。當代理互動歷史中的最後一則訊息包含「saved」一詞時，代理將會終止。這在代理的任務一旦收到「已儲存」的確認後即視為完成的情境中非常有用。定義互動方法。在預約計畫儲存後，當接收到 saved 訊號時即可停止。


In [ ]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "saved" in history[-1].content.lower()

# 說明：

這行程式碼透過呼叫 create() 方法，使用預設或預先定義的設定來初始化一個 AzureAIAgentSettings 物件。此設定物件（ai_agent_settings）接著可以用來配置和管理 AzureAIAgent 實例。


In [ ]:
ai_agent_settings = AzureAIAgentSettings.create()

# 解釋：
透過匯入 requests 函式庫，您可以輕鬆地在 Python 程式中發出 HTTP 請求並與網路服務互動。


In [ ]:
import requests

# 說明：
這是一個用來儲存用於存取 SERP（搜尋引擎結果頁）API 服務的 API 金鑰的變數。API 金鑰是一組獨特的識別碼，用於驗證與您的帳戶相關聯的請求。

'GOOGLE_SEARCH_API_KEY'：這是一個佔位字串。您需要將 ''GOOGLE_SEARCH_API_KEY' 替換成您的實際 SERP API 金鑰。

目的：此行的目的是將 API 金鑰儲存在變數中，以便用來驗證對 SERP API 服務的請求。API 金鑰是存取該服務並執行搜尋所必需的。

如何取得 SERP API 金鑰：要取得 SERP API 金鑰，請依照 https://serpapi.com 的一般步驟操作（實際步驟可能依您使用的特定 SERP API 服務而異）：

選擇一個 SERP API 服務：市面上有多種 SERP API 服務，例如 SerpAPI、Google Custom Search JSON API 等。請選擇最符合您需求的服務。

註冊帳戶：

前往所選擇的 SERP API 服務網站 https://www.serpapi.com 並註冊帳戶。您可能需要提供一些基本資訊並驗證您的電子郵件信箱。

建立 API 金鑰：

註冊完成後，登入帳戶並前往 API 區域或儀表板。尋找建立或產生新 API 金鑰的選項。

複製 API 金鑰：

當 API 金鑰產生後，將其複製。該金鑰將用來驗證您對 SERP API 服務的請求。

替換佔位字串：

將您 .env 檔案中的佔位字串替換成您的 API 金鑰。


In [ ]:
SERPAPI_SEARCH_API_KEY=os.getenv('SERPAPI_SEARCH_API_KEY')

In [ ]:
SERPAPI_SEARCH_ENDPOINT = os.getenv('SERPAPI_SEARCH_ENDPOINT')

# 說明：
BookingPlugin 類別提供使用 Serpapi.com Google 搜索 API 預訂飯店與航班的方法。它構建所需的參數、發送 API 請求，並處理回應以返回相關的預訂資訊。API 金鑰（SERPAPI_SEARCH_API_KEY）和端點（SERPAPI_SEARCH_ENDPOINT）用於驗證和發送請求至 Google 搜索 API。


In [ ]:
# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""
    @kernel_function(description="booking hotel")
    def booking_hotel(self,query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-in Time"])-> Annotated[str, "Return the result of booking hotel infomation"]:

        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "2",
            "currency": "USD",
            "gl": "us",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY
        }

        response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=params)
        if response.status_code == 200:
            response = response.json()
            return response["properties"]
        else:
            return None

    
    @kernel_function(description="booking fight")
    def  booking_fight(self,origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"])-> Annotated[str, "Return the result of booking fight infomation"]:
        
        go_params = {
            "engine": "google_flights",   
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }

        print(go_params)

        go_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=go_params)


        result = ''

        if go_response.status_code == 200:
            response = go_response.json()

            result += "# outbound \n " + str(response)
        else:
            print('error!!!')
            # return None

        
        back_params = {
            "engine": "google_flights",   
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": return_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }


        print(back_params)


        back_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=back_params)



        if back_response.status_code == 200:
            response = back_response.json()

            result += "\n # return \n"  + str(response)

        else:
            print('error!!!')
            # return None
        
        print(result)

        return result

        


# 說明：
SavePlugin 類別提供了一個方法 saving_plan，用於使用 Azure AI 服務保存旅遊計劃。它設置 Azure 憑證，創建一個 AI 代理，處理使用者輸入以生成並保存旅遊計劃內容，並處理檔案保存和清理操作。該方法成功完成後會返回「Saved」。


In [ ]:
class SavePlugin:
    """Save Plugin for customers"""
    @kernel_function(description="saving plan")
    async def saving_plan(self,tripplan: Annotated[str, "The content of trip plan"])-> Annotated[str, "Return status of save content"]:

        async with (
            DefaultAzureCredential() as creds,
            AzureAIAgent.create_client(
                credential=creds,
                conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
            ) as client,
        ):

            code_interpreter = CodeInterpreterTool()
            
            agent_definition = await client.agents.create_agent(
                model=ai_agent_settings.model_deployment_name,
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources,
            )


            agent = AzureAIAgent(
                client=client,
                definition=agent_definition,
            )

            thread = await client.agents.create_thread()


            user_inputs = [
                """
            
                        You are my Python programming assistant. Generate code,save """+ tripplan +
                        
                    """    
                        and execute it according to the following requirements

                        1. Save blog content to trip-{YYMMDDHHMMSS}.md

                        2. give me the download this file link
                    """
            ]



            try:
                for user_input in user_inputs:
                    # Add the user input as a chat message
                    await agent.add_chat_message(
                        thread_id=thread.id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
                    )
                    print(f"# User: '{user_input}'")
                    # Invoke the agent for the specified thread
                    async for content in agent.invoke(thread_id=thread.id):
                        if content.role != AuthorRole.TOOL:
                            print(f"# Agent: {content.content}")

                    
                    messages = await client.agents.list_messages(thread_id=thread.id)

                    # OpenAIPageableListOfThreadMessage
                    # OpenAIPageableListOfThreadMessage


                    for file_path_annotation in messages.file_path_annotations:

                            file_name = os.path.basename(file_path_annotation.text)

                            await client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./trip")

                    
            finally:
                await client.agents.delete_thread(thread.id)
                await client.agents.delete_agent(agent.id)


        return "Saved"

# 說明：
此程式碼設置了 Azure AI 代理，用於處理航班和飯店預訂，以及根據使用者輸入儲存行程計畫。它使用 Azure 認證來建立和配置代理，透過群組聊天處理使用者輸入，並在任務完成後確保適當清理。代理使用特定的外掛程式（BookingPlugin 和 SavePlugin）來執行各自的任務。


In [ ]:
async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):
    BOOKING_AGENT_NAME = "BookingAgent"
    BOOKING_AGENT_INSTRUCTIONS = """
    You are a booking agent. Help me book flights or hotels.

    Thought: Please understand the user's intention and confirm whether to use the reservation system to complete the task.

    Actions:
    - For flight bookings, convert the departure and destination names into airport codes.
    - Use the appropriate API for hotel or flight bookings. Verify that all necessary parameters are available. If any parameters are missing, ask the user to provide them. If all parameters are complete, call the corresponding function.
    - If the task is not related to hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
      - For flight bookings, output separate outbound and return contents in the order of:
        Departure Airport | Airline | Flight Number | Departure Time | Arrival Airport | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
      - For hotel bookings, output in the order of:
        Property Name | Property Description | Check-in Time | Check-out Time | Prices | Nearby Places | Hotel Class | GPS Coordinates.
    """

    SAVE_AGENT_NAME = "SaveAgent"
    SAVE_AGENT_INSTRUCTIONS = """
    You are a save tool agent. Help me to save the trip plan.
    """

    # Create agent definition
    booking_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=BOOKING_AGENT_NAME,
        instructions=BOOKING_AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent
    booking_agent = AzureAIAgent(
        client=client,
        definition=booking_agent_definition,
        # Optionally configure polling options
        # polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Add the sample plugin to the kernel
    booking_agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking")

    # Create agent definition
    save_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=SAVE_AGENT_NAME,
        instructions=SAVE_AGENT_INSTRUCTIONS
    )

    # Create the AzureAI Agent
    save_agent = AzureAIAgent(
        client=client,
        definition=save_agent_definition,
    )

    save_agent.kernel.add_plugin(SavePlugin(), plugin_name="saving")

    user_inputs = [
        "I have a business trip from London to New York in Feb 20 2025 to Feb 27 2025 ,help me to book a hotel and fight tickets and save it"
    ]

    chat = AgentGroupChat(
        agents=[booking_agent, save_agent],
        termination_strategy=ApprovalTerminationStrategy(agents=[save_agent], maximum_iterations=10),
    )

    try:
        for user_input in user_inputs:
            # Add the user input as a chat message
            await chat.add_chat_message(
                ChatMessageContent(role=AuthorRole.USER, content=user_input)
            )
            print(f"# User: '{user_input}'")

            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

            print(f"# IS COMPLETE: {chat.is_complete}")

            print("*" * 60)
            print("Chat History (In Descending Order):\n")
            async for message in chat.get_chat_messages(agent=save_agent):
                print(f"# {message.role} - {message.name or '*'}: '{message.content}'")
    finally:
        await chat.reset()
        await client.agents.delete_agent(save_agent.id)
        await client.agents.delete_agent(booking_agent.id)


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**免責聲明**：  
本文件係使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。雖然我們致力於確保準確性，但請注意自動翻譯可能包含錯誤或不精確之處。文件的原始語言版本應視為權威來源。對於重要資訊，建議採用專業人工翻譯。我們不對因使用本翻譯所導致的任何誤解或誤釋承擔責任。
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
